In [53]:
# Defined function to parse the Pineapple logs of collected devices

from datetime import date
import re, time, json
import pandas as pd

def parsePineapple(log):
    parsed = {}
    months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04',
              'May':'05','Jun':'06','Jul':'07','Aug':'08',
              'Sep':'09','Oct':'10','Nov':'11','Dec':'12'}

    k_probe = '^([A-Z][a-z]{2}) +?(\d{1,2}) +?(\d{2}):(\d{2}):(\d{2}) +?(Probe Request from) +?([0-9a-f:]{17}) +?for SSID \'([^\']+)\''
    m = re.match(k_probe, log)
    
    k_association = '^([A-Z][a-z]{2}) +?(\d{1,2}) +?(\d{2}):(\d{2}):(\d{2}) +?([0-9a-f:]{17}) +?(trying to associate with) \'([^\']+)\''
    n = re.match(k_association, log)
    if m:
        parsed['log_type'] = 'pineAP'
        parsed['dtoi'] = str('2015' + months[m.group(1)] + m.group(2).zfill(2) + m.group(3) + m.group(4) + m.group(5))
        parsed['producer'] = '01c6191c-1e5d-4107-a7bc-ad74f53a51ea'
        parsed['subject'] = m.group(7).replace(':','')
        parsed['subject_class'] = 'macAddress'
        parsed['verb'] = 'probe'
        parsed['object'] = m.group(8)
        parsed['object_class'] = 'SSID'
    if n:
        parsed['log_type'] = 'pineAP'
        parsed['dtoi'] = str('2015' + months[n.group(1)] + n.group(2).zfill(2) + n.group(3) + n.group(4) + n.group(5))
        parsed['producer'] = '01c6191c-1e5d-4107-a7bc-ad74f53a51ea'
        parsed['subject'] = n.group(6).replace(':','')
        parsed['subject_class'] = 'macAddress'
        parsed['verb'] = 'association request'
        parsed['object'] = n.group(8)
        parsed['object_class'] = 'SSID'
    else:
        pass
    return parsed


In [6]:
#RawData = read lines from the pineapple text file

with open('../../Desktop/pineapple_logs.log') as rawdata:
    rawData = rawdata.readlines()

#What the collected data looks like prior to parsing
for i in range(5):
    print(rawData[i])
    
#Create a new dataList to append the parsed pineapple text file
dataList = []
errorDataset = []
for line in rawData:
    try:
        dataList.append(parsePineapple(line))
    except:
        errorDataset.append(line)

print("Number of collected devices: ",len(dataList))
print("Errors: " + str(len(errorDataset)))


#print(dataList[2])
        

Number of collected devices:  0
Errors: 39487


In [4]:
df = pd.DataFrame(dataList)
df.head(2)


In [2]:
df.describe()

In [1]:
#List unique values in the df['object'] column
pd.unique(df.object.ravel())

In [58]:
#Mask Personal SSID
df.object = df.object.replace('Malware_Partition', 'RegNet')
df.object = df.object.replace('PattySue', 'DeguzmanNet')


In [63]:
#Mask Personal mac addresses

def random_mac():
    hexchars = ['1','2','3','4','5','6','7','8','9','0','a','b','c','d','e','f']
    random_mac = ''
    for digit in range(12):
        random_mac += random.choice(hexchars)
    return random_mac

for mac in existing_macs:
    df.subject = df.subject.replace(mac, random_mac())

df.head()


,dtoi,log_type,object,object_class,producer,subject,subject_class,verb
0,20150911194354,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe
1,20150906094221,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,a38ac78b45f8,macAddress,probe
2,20150908104738,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,44c162d68b54,macAddress,probe
3,20150907070321,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe
4,20150912074547,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe


In [64]:
df


,dtoi,log_type,object,object_class,producer,subject,subject_class,verb
0,20150911194354,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe
1,20150906094221,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,a38ac78b45f8,macAddress,probe
2,20150908104738,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,44c162d68b54,macAddress,probe
3,20150907070321,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe
4,20150912074547,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,c9c19a186c85,macAddress,probe
5,20150906210304,pineAP,DeguzmanNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,e5e5ba02ea28,macAddress,probe
6,20150906195324,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,a38ac78b45f8,macAddress,probe
7,20150903105400,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,a38ac78b45f8,macAddress,probe
8,20150912145315,pineAP,DeguzmanNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,e5e5ba02ea28,macAddress,probe
9,20150912145322,pineAP,RegNet,SSID,01c6191c-1e5d-4107-a7bc-ad74f53a51ea,51e76b23a7c7,macAddress,probe


In [65]:
#Export to CSV file for everyone to see in GitHUB. 
df.to_csv('Pineapple_Logs.csv')